In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile

# Extração e Carga de Dados do CNEFE 2022

## Apresentação

Este notebook descreve o processo de download, extração e unificação dos arquivos CSV contendo os registros de endereços de todas as Unidades Federativas (UF) do Brasil, presentes no Cadastro Nacional de Endereços para Fins Estatísticos (CNEFE) do IBGE (Instituto Brasileiro de Geografia e Estatística). Esses dados foram coletados durante o Censo Demográfico de 2022 e são fundamentais para diversas análises estatísticas e geográficas.

**Objetivos:**
1. Realizar o download dos arquivos de endereços de cada UF.
2. Verificar a integridade e atualizar os arquivos, se necessário.
3. Descompactar os arquivos ZIP.
4. Unificar todos os registros em um único arquivo CSV para facilitar o processamento e análise subsequente.

Os arquivos estão disponíveis para download no site do IBGE, no link abaixo:

[CNEFE - Cadastro Nacional de Endereços para Fins Estatísticos](https://www.ibge.gov.br/estatisticas/sociais/populacao/38734-cadastro-nacional-de-enderecos-para-fins-estatisticos.html?=&t=downloads)


Inicialmente, é necessário informar o diretório que se pretende salvar os arquivos desse processo de extração e carga:

In [ ]:
# Definir o diretório de destino dos arquivos de endereço do CNEFE
diretorio_enderecos_uf = input("Insira o diretório de destino dos arquivos a serem baixados: ")

## Extração dos Arquivos

Nesta seção, vamos baixar os arquivos ZIP contendo os registros de endereços de cada UF. O processo envolve as seguintes etapas:

1. **Obter os Links dos Arquivos:** Navegar na página do IBGE e obter os links para os arquivos ZIP de cada UF.
2. **Verificar a Existência e Atualização dos Arquivos Locais:** Antes de baixar, verificamos se o arquivo já existe localmente e se está atualizado comparando o tamanho do arquivo remoto.
3. **Download dos Arquivos:** Se o arquivo não existir ou estiver desatualizado, baixamos o arquivo e salvamos no diretório especificado.

Abaixo estão as funções necessárias para realizar essas tarefas.

### `obter_links(url_base)`

**Descrição:**
Esta função realiza uma requisição GET para uma URL base que contém links para arquivos de endereços por UF do CNEFE. Utiliza BeautifulSoup para parsear o HTML da página e extrair os links que terminam com '.zip'.

**Parâmetros:**
- `url_base` (str): URL base de onde serão obtidos os links.

**Retorno:**
- Retorna uma lista de URLs completas dos arquivos ZIP encontrados na página.


In [3]:
# Obtém todos os links dos arquivos ZIP disponíveis na URL base.
def obter_links(url_base):

    # Faz uma requisição GET para a URL base
    response = requests.get(url_base)
    # Verifica se a requisição foi bem-sucedida
    response.raise_for_status()
    # Parseia o conteúdo HTML para extrair os links dos arquivos
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Cria e retorna uma lista de URLs dos arquivos ZIP
    return [url_base + link.get('href') for link in soup.find_all('a') if link.get('href') and link.get('href').endswith('.zip')]

### `verificar_arquivo(url_arquivo, caminho_arquivo)`

**Descrição:**
Verifica se um arquivo local existe e se seu tamanho corresponde ao tamanho do arquivo remoto (para verificar se o arquivo local está atualizado).

**Parâmetros:**
- `url_arquivo` (str): URL do arquivo remoto.
- `caminho_arquivo` (str): Caminho do arquivo local.

**Retorno:**
- Retorna `True` se o arquivo local existe e está atualizado, caso contrário, retorna `False`.


In [4]:
# Verifica se o arquivo local existe e se o tamanho corresponde ao tamanho do arquivo remoto.
def verificar_arquivo(url_arquivo, caminho_arquivo):
    
    # Faz uma requisição HEAD para obter os metadados do arquivo remoto
    response = requests.head(url_arquivo)
    # Verifica se a requisição foi bem-sucedida
    response.raise_for_status()
    
    # Obtém o tamanho do arquivo remoto a partir dos cabeçalhos da resposta
    tamanho_arquivo_remoto = int(response.headers.get('Content-Length', 0))
    
    # Verifica se o arquivo já existe localmente
    if os.path.exists(caminho_arquivo):
        # Obtém o tamanho do arquivo local
        tamanho_arquivo_local = os.path.getsize(caminho_arquivo)
        # Compara os tamanhos do arquivo local e remoto
        if tamanho_arquivo_local == tamanho_arquivo_remoto:
            return True
        else:
            print(f"{os.path.basename(caminho_arquivo)} sofreu modificações na base original! O download será feito novamente.")
    return False

### `baixar_enderecos_uf(url_arquivo, caminho_arquivo)`

**Descrição:**
Faz o download de um arquivo a partir de uma URL e salva-o no caminho especificado.

**Parâmetros:**
- `url_arquivo` (str): URL do arquivo a ser baixado.
- `caminho_arquivo` (str): Caminho onde o arquivo será salvo localmente.

In [5]:
# Faz o download do arquivo a partir da URL e salva no caminho especificado.
def baixar_enderecos_uf(url_arquivo, caminho_arquivo):

    # Faz uma requisição GET para baixar o arquivo
    response = requests.get(url_arquivo)
    # Verifica se a requisição foi bem-sucedida
    response.raise_for_status()
    
    # Salva o conteúdo do arquivo no caminho especificado
    with open(caminho_arquivo, 'wb') as file:
        file.write(response.content)
    print(f"{os.path.basename(caminho_arquivo)} salvo em {caminho_arquivo}")

### `extrair_arquivos(url_base, diretorio)`

**Descrição:**
Realiza o download de todos os arquivos de endereços por UF disponíveis na `url_base`, verifica se já existem localmente e, se necessário, faz o download e salva-os no diretório especificado.

**Parâmetros:**
- `url_base` (str): URL base que contém os links para os arquivos de endereços por UF.
- `diretorio` (str): Diretório onde os arquivos serão salvos.

In [6]:
# Busca todos os arquivos do CNEFE disponíveis na URL base e salva no diretório especificado.
def extrair_arquivos(url_base, diretorio):

    # Obtém a lista de URLs dos arquivos ZIP
    urls_arquivos = obter_links(url_base)
    
    # Inicializa o contador de arquivos 
    conta_arquivos = 0   
    
    # Itera sobre cada URL de arquivo ZIP
    for url_arquivo in urls_arquivos:
        # Extrai o nome do arquivo da URL
        nome_arquivo = os.path.basename(url_arquivo)
        # Cria o caminho completo do arquivo local
        caminho_arquivo = os.path.join(diretorio, nome_arquivo)
        
        # Verifica se o arquivo local existe e se está atualizado
        if verificar_arquivo(url_arquivo, caminho_arquivo):
            # Informa que o arquivo já existe e está atualizado
            print(f"{nome_arquivo} já existe!")
        else:
            # Se o arquivo não existe ou está desatualizado, faz o download
            print(f"Baixando {nome_arquivo}...")
            baixar_enderecos_uf(url_arquivo, caminho_arquivo)
        
        # Inicializa o contador de arquivos 
        conta_arquivos += 1   

    print(f"Todos os {conta_arquivos} arquivos de enderecos por UF foram baixados.")

In [ ]:
# Cria diretório a parte para alocar o arquivo contendo todos os registros de enderecos do CNEFE
enderecos_unificados = '/base_unificada'
if not os.path.exists(diretorio_enderecos_uf + enderecos_unificados):
    os.makedirs(diretorio_enderecos_uf + enderecos_unificados)
    print(f"Diretório {diretorio_enderecos_uf + enderecos_unificados} criado!")
    
# Caminho para o arquivo que conterá todos os registros de endereços das UFs do CNEFE
arquivo_enderecos_br = os.path.join(diretorio_enderecos_uf, diretorio_enderecos_uf + enderecos_unificados + '/enderecos_br.csv')

# URL base para download dos arquivos de enderecos por UF compactados 
url_base = "https://ftp.ibge.gov.br/Cadastro_Nacional_de_Enderecos_para_Fins_Estatisticos/Censo_Demografico_2022/Arquivos_CNEFE/UF/"

# Baixar os arquivos compactados fornecidos pelo IBGE
extrair_arquivos(url_base, diretorio_enderecos_uf)

### `descompactar_arquivos(diretorio)`

**Descrição:**
Descompacta todos os arquivos de endereços por UF encontrados em um diretório específico.

**Parâmetros:**
- `diretorio` (str): Diretório onde estão localizados os arquivos de endereços por UF que serão descompactados.

In [8]:
# Descompacta todos os arquivos no diretório especificado.
def descompactar_arquivos(diretorio):

    # Inicializa o contador de arquivos 
    conta_arquivos = 0
    
    # Lista todos os arquivos ZIP no diretório especificado
    arquivos_cnefe = [f for f in os.listdir(diretorio) if f.endswith('.zip')]
    
    # Itera sobre cada arquivo ZIP encontrado
    for arquivo_cnefe in arquivos_cnefe:
        diretorio_cnefe = os.path.join(diretorio, arquivo_cnefe)
        print(f"Descompactando {arquivo_cnefe}...")
        
        # Abre o arquivo ZIP no modo de leitura        
        with ZipFile(diretorio_cnefe, 'r') as zip_ref:
            # Extrai todo o conteúdo do arquivo ZIP para o diretório especificado
            zip_ref.extractall(diretorio)
        
        # Incrementa o contador de arquivos    
        conta_arquivos += 1
        
        print(f"{arquivo_cnefe} descompactado.")

    print(f"Foram descompactados {conta_arquivos} arquivos contendo enderecos de UFs.")


# Descompactar arquivos
descompactar_arquivos(diretorio_enderecos_uf)

Descompactando 41_PR.zip...
41_PR.zip descompactado.
Descompactando 21_MA.zip...
21_MA.zip descompactado.
Descompactando 51_MT.zip...
51_MT.zip descompactado.
Descompactando 13_AM.zip...
13_AM.zip descompactado.
Descompactando 52_GO.zip...
52_GO.zip descompactado.
Descompactando 15_PA.zip...
15_PA.zip descompactado.
Descompactando 31_MG.zip...
31_MG.zip descompactado.
Descompactando 29_BA.zip...
29_BA.zip descompactado.
Descompactando 24_RN.zip...
24_RN.zip descompactado.
Descompactando 12_AC.zip...
12_AC.zip descompactado.
Descompactando 43_RS.zip...
43_RS.zip descompactado.
Descompactando 53_DF.zip...
53_DF.zip descompactado.
Descompactando 27_AL.zip...
27_AL.zip descompactado.
Descompactando 25_PB.zip...
25_PB.zip descompactado.
Descompactando 17_TO.zip...
17_TO.zip descompactado.
Descompactando 35_SP.zip...
35_SP.zip descompactado.
Descompactando 50_MS.zip...
50_MS.zip descompactado.
Descompactando 42_SC.zip...
42_SC.zip descompactado.
Descompactando 22_PI.zip...
22_PI.zip descompa

## Concatenação e Carga dos Arquivos

Após realizar a extração dos arquivos de endereços por estado da base do CNEFE, será realizada a junção desses arquivos em um só arquivo csv, contendo os registros de endereços de todas as UFs do Brasil.

### `criar_base_unificada(diretorio, arquivo_final, batch_size=1000)`

**Descrição:**
Lê todos os arquivos de endereços por UF de um diretório especificado, combina-os em um único arquivo de enderecos_br e salva-o. Utiliza o Pandas para ler os arquivos em lotes (chunks) para gerenciar a memória de forma eficiente.

**Parâmetros:**
- `diretorio` (str): Diretório onde estão localizados os arquivos de endereços por UF a serem combinados.
- `arquivo_enderecos_br` (str): Caminho completo do arquivo de enderecos_br.
- `batch_size` (int, opcional): Tamanho do lote para leitura dos arquivos de endereços (padrão: 1000).

In [ ]:
# Une todos os arquivos de endereço por UF do CNEFE um único arquivo contendo os enderecos de todo o Brasil.
def criar_base_unificada(diretorio, arquivo_enderecos_br, batch_size=1000):

    # Lista todos os arquivos CSV no diretório especificado
    arquivos_enderecos_uf = [f for f in os.listdir(diretorio) if f.endswith('.csv')]
    
    # Variável para controlar a escrita do cabeçalho no arquivo final
    cabecalho = True
    
    # Inicializa contador de arquivos
    conta_arquivos = 0 
    
    # Itera sobre cada arquivo CSV encontrado
    for arquivo in arquivos_enderecos_uf:
        caminho_completo = os.path.join(diretorio, arquivo)  # Cria o caminho completo do arquivo
        print(f"Lendo o arquivo: {caminho_completo}")
        
        # Lê o arquivo CSV em lotes para evitar sobrecarga de memória
        for batch in pd.read_csv(caminho_completo, sep=';', encoding='utf-8', low_memory=False, chunksize=batch_size):
            # Escreve cada lote no arquivo CSV final
            batch.to_csv(arquivo_enderecos_br, mode='a', header=cabecalho, index=False, sep=';', encoding='utf-8')
            # Após o primeiro lote, define `cabecalho` como False para não repetir o cabeçalho
            cabecalho = False
            
        # Incrementa o contador de arquivos
        conta_arquivos += 1 
    
    print(f"Foram concatenadas {conta_arquivos} bases de enderecos de UFs.")
    print(f"Base de Endereços do Brasil salva em: {arquivo_enderecos_br}")
    
# Unir arquivos
criar_base_unificada(diretorio_enderecos_uf, arquivo_enderecos_br)

### `remover_enderecos_uf(diretorio, arquivo_final)`

**Descrição:**
Deleta todos os arquivos de endereços por Unidade Federativa (UF) no diretório especificado, exceto o arquivo final especificado.

**Parâmetros:**
- `diretorio` (str): Diretório onde estão localizados os arquivos de endereco a serem deletados.
- `arquivo_enderecos_br` (str): Caminho completo do arquivo final que não será deletado.

In [10]:
# Deleta todos os arquivos de endereço por UF no diretório.
def remover_enderecos_uf(diretorio, arquivo_enderecos_br):
    
    # Lista todos os arquivos de enderecos por UF no diretório especificado
    arquivos_enderecos_uf = [f for f in os.listdir(diretorio) if f.endswith('.csv') and f != os.path.basename(arquivo_enderecos_br)]
    
    # Inicializa contador de arquivos
    conta_arquivos = 0 
    
    # Deleta cada arquivo individual na lista
    for arquivo in arquivos_enderecos_uf:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        os.remove(caminho_arquivo)
        print(f"Arquivo {arquivo} deletado.")
    
        # Incrementa contador de arquivos
        conta_arquivos += 1 
    
    print(f"Foram deletados os {conta_arquivos} arquivos csv de endereços por UF.")
        
# Deletar os arquivos
remover_enderecos_uf(diretorio_enderecos_uf, arquivo_enderecos_br)

Arquivo 21_MA.csv deletado.
Arquivo 15_PA.csv deletado.
Arquivo 33_RJ.csv deletado.
Arquivo 23_CE.csv deletado.
Arquivo 13_AM.csv deletado.
Arquivo 51_MT.csv deletado.
Arquivo 42_SC.csv deletado.
Arquivo 16_AP.csv deletado.
Arquivo 12_AC.csv deletado.
Arquivo 14_RR.csv deletado.
Arquivo 35_SP.csv deletado.
Arquivo 28_SE.csv deletado.
Arquivo 17_TO.csv deletado.
Arquivo 29_BA.csv deletado.
Arquivo 26_PE.csv deletado.
Arquivo 43_RS.csv deletado.
Arquivo 27_AL.csv deletado.
Arquivo 24_RN.csv deletado.
Arquivo 11_RO.csv deletado.
Arquivo 53_DF.csv deletado.
Arquivo 50_MS.csv deletado.
Arquivo 31_MG.csv deletado.
Arquivo 22_PI.csv deletado.
Arquivo 32_ES.csv deletado.
Arquivo 52_GO.csv deletado.
Arquivo 25_PB.csv deletado.
Arquivo 41_PR.csv deletado.
Foram deletados os 27 arquivos csv de endereços por UF.


## Resultado

Ao final da execução, foi possível gerar um arquivo consolidado que facilita a análise e o processamento dos dados de endereços do Brasil para o Censo 2022. Este arquivo pode ser usado para diversas análises estatísticas e geográficas.

Em carater de observação, foi gerado um dataframe com os registros de endereços iniciais e finais do arquivo csv que contém os endereços de todas as UFs concatenados. A partir desses registros, é possível verificar se a carga ocorreu conforme esperado.

O arquivo CSV final contendo todos os endereços foi salvo em:

In [ ]:
print(arquivo_enderecos_br)

### `gerar_enderecos_extremos(arquivo_enderecos_br, n_primeiros=100, n_ultimos=100)`

**Descrição:**
Esta função lê os primeiros e últimos registros do arquivo de enderecos_br, lidando de forma eficiente com grandes volumes de dados.

**Parâmetros:**
- `arquivo_enderecos_br` (str): Caminho do arquivo que contém os registros de endereços.
- `n_primeiros` (int, opcional): Número de primeiros registros a serem lidos. O padrão é 100.
- `n_ultimos` (int, opcional): Número de últimos registros a serem lidos. O padrão é 100.

**Retorno:**
- `primeiros_enderecos` (DataFrame): DataFrame contendo os primeiros registros lidos do arquivo CSV.
- `ultimos_enderecos` (DataFrame): DataFrame contendo os últimos registros lidos do arquivo CSV.


In [12]:
# Gera dataframes contendo os primeiros e os últimos registros do arquivo de endereços.

def gerar_enderecos_extremos(arquivo_enderecos_br, n_primeiros=100, n_ultimos=100):

    # Leitura dos primeiros registros diretamente
    primeiros_enderecos = pd.read_csv(arquivo_enderecos_br, sep=';', encoding='utf-8', nrows=n_primeiros)

    # Determina o número total de linhas no arquivo sem carregar todo o conteúdo na memória
    with open(arquivo_enderecos_br, 'r', encoding='utf-8') as f:
        total_linhas = sum(1 for _ in f)

    # Calcula quantas linhas devem ser puladas para obter os últimos registros
    skip_linhas = max(1, total_linhas - n_ultimos)

    # Leitura dos últimos registros
    ultimos_enderecos = pd.read_csv(arquivo_enderecos_br, sep=';', encoding='utf-8', skiprows=skip_linhas, names=primeiros_enderecos.columns, header=None)
    
    # Retorna os DataFrames dos primeiros e últimos registros
    return primeiros_enderecos, ultimos_enderecos

# Gerar DataFrame com os registros iniciais e finais
primeiros_enderecos, ultimos_enderecos = gerar_enderecos_extremos(arquivo_enderecos_br)

In [13]:
# Exibir primeiros registros
primeiros_enderecos.head()

,COD_UNICO_ENDERECO,COD_UF,COD_MUNICIPIO,COD_DISTRITO,COD_SUBDISTRITO,COD_SETOR,NUM_QUADRA,NUM_FACE,CEP,DSC_LOCALIDADE,...,VAL_COMP_ELEM5,LATITUDE,LONGITUDE,NV_GEO_COORD,COD_ESPECIE,DSC_ESTABELECIMENTO,COD_INDICADOR_ESTAB_ENDERECO,COD_INDICADOR_CONST_ENDERECO,COD_INDICADOR_FINALIDADE_CONST,COD_TIPO_ESPECI
0,221696317,21,2106102,210610205,21061020500,210610205000027P,18,2,65895000,AGROVILA,...,NaN,-7.464288,-45.379863,1,7,NaN,NaN,1.0,1.0,NaN
1,221696319,21,2106102,210610205,21061020500,210610205000027P,18,2,65895000,AGROVILA,...,NaN,-7.464368,-45.379811,1,7,NaN,NaN,1.0,1.0,NaN
2,221696323,21,2106102,210610205,21061020500,210610205000027P,18,2,65895000,AGROVILA,...,NaN,-7.464543,-45.379672,1,1,NaN,NaN,NaN,NaN,101.0
3,221696325,21,2106102,210610205,21061020500,210610205000027P,18,2,65895000,AGROVILA,...,NaN,-7.464607,-45.379636,1,1,NaN,NaN,NaN,NaN,101.0
4,221696328,21,2106102,210610205,21061020500,210610205000027P,18,2,65895000,AGROVILA,...,NaN,-7.464769,-45.379480,1,1,NaN,NaN,NaN,NaN,101.0


In [14]:
# Exibir ultimos registros
ultimos_enderecos.tail()

,COD_UNICO_ENDERECO,COD_UF,COD_MUNICIPIO,COD_DISTRITO,COD_SUBDISTRITO,COD_SETOR,NUM_QUADRA,NUM_FACE,CEP,DSC_LOCALIDADE,...,VAL_COMP_ELEM5,LATITUDE,LONGITUDE,NV_GEO_COORD,COD_ESPECIE,DSC_ESTABELECIMENTO,COD_INDICADOR_ESTAB_ENDERECO,COD_INDICADOR_CONST_ENDERECO,COD_INDICADOR_FINALIDADE_CONST,COD_TIPO_ESPECI
95,96172114,41,4106902,410690205,41069020506,410690205060162P,1,4,80320290,VILA IZABEL,...,NaN,-25.456177,-49.295490,1,1,NaN,NaN,NaN,NaN,101.0
96,96172126,41,4106902,410690205,41069020506,410690205060162P,1,4,80320290,VILA IZABEL,...,NaN,-25.455806,-49.295505,2,1,NaN,NaN,NaN,NaN,103.0
97,206529005,41,4118402,411840205,41184020500,411840205000050P,3,2,87709060,VILA CITY,...,NaN,-23.082063,-52.478741,1,1,NaN,NaN,NaN,NaN,101.0
98,206527971,41,4119905,411990505,41199050500,411990505000175P,5,1,84070120,ORFAS,...,NaN,-25.083346,-50.169226,1,6,X5ADMINISTRADORA E CORRETORA DE SEGURIS,2.0,NaN,NaN,NaN
99,65443363,41,4106902,410690205,41069020509,410690205090148P,3,2,81450026,CIDADE INDUSTRIAL DE CURITIBA,...,NaN,-25.499557,-49.348389,1,1,NaN,NaN,NaN,NaN,101.0
